# Using SQLite with Python

SQLite is a lightweight and embedded relational database management system that is easy to use and requires minimal configuration. It is a popular choice for small to medium-sized applications, especially when portability and simplicity are essential. Python provides a built-in module called `sqlite3` that allows you to interact with SQLite databases seamlessly.

In this guide, we'll cover the following topics:

1. Installing SQLite and Python
2. Connecting to a SQLite database
3. Creating tables and inserting data
4. Performing CRUD operations (Create, Read, Update, Delete)
5. Executing advanced queries (Joins, Aggregates, Subqueries)

Let's dive into the details:

## 0. Datatypes in SQLite

In SQLite, data types determine the type of value that can be stored in each column of a table. SQLite is a dynamically typed database, which means it is more flexible with data types compared to traditional statically typed databases. Here are the main data types used in SQLite:

1. **NULL**: A special data type representing a NULL value (empty or missing data).

2. **INTEGER**: An integer value (whole numbers) stored in 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude.

3. **REAL**: A floating-point value used to store floating-point numbers like decimal numbers.

4. **TEXT**: A text string, which can store any alphanumeric characters, dates, or strings.

5. **BLOB**: Binary Large Object, used to store binary data such as images, documents, etc.

6. **NUMERIC**: This data type can store any type of numeric value (integer or floating-point) as well as dates and timestamps.

SQLite is not strict with data types, which means you can store data of different types in the same column. SQLite uses dynamic typing to determine the affinity of a value stored in a column based on the value's content.

For example, if you insert an integer value into a column, SQLite will assign INTEGER affinity to that column, but you can still insert a text value in the same column, and SQLite will convert it automatically.

Here's a basic example of creating a table with different data types:

```sql
CREATE TABLE my_table (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    height REAL,
    bio BLOB
);
```

In the example above, we have a table named `my_table` with columns of different data types: `id` (INTEGER), `name` (TEXT), `age` (INTEGER), `height` (REAL), and `bio` (BLOB).

It's important to note that while SQLite is flexible with data types, it is still recommended to follow good database design practices and use appropriate data types for each column to ensure data integrity and consistency.

## 1. Installing SQLite and Python

SQLite is already included in the Python standard library, so you don't need to install anything separately.

Make sure you have Python installed on your system. If not, download and install Python from the official website (https://www.python.org/downloads/).

## 2. Connecting to a SQLite database

To work with SQLite in Python, you need to import the `sqlite3` module and connect to the database. If the specified database does not exist, it will be created.

```python
import sqlite3

# Connect to a database (if it doesn't exist, it will be created)
conn = sqlite3.connect('example.db')

# Create a cursor object to interact with the database
cursor = conn.cursor()
```

## 3. Creating tables and inserting data

You need to define the table schema before inserting data. A schema includes the table name, column names, and their data types.

```python
# Create a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        age INTEGER
    )
''')

# Insert data into the table
cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ('John Doe', 30))
cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ('Jane Smith', 25))

# Save changes to the database
conn.commit()
```

## 4. Performing CRUD operations

### 4.1. Read data (SELECT)

```python
# SELECT all rows
cursor.execute('SELECT * FROM users')
rows = cursor.fetchall()  # Fetch all rows
for row in rows:
    print(row)

# SELECT with conditions
cursor.execute('SELECT * FROM users WHERE age > ?', (25,))
rows = cursor.fetchall()
```

### 4.2. Update data (UPDATE)

```python
# Update data
cursor.execute('UPDATE users SET age = ? WHERE name = ?', (35, 'John Doe'))
conn.commit()
```

### 4.3. Insert data (CREATE)

```python
# Insert data into the table
cursor.execute('INSERT INTO users (name, age) VALUES (?, ?)', ('Mary Johnson', 28))
conn.commit()
```

### 4.4. Delete data (DELETE)

```python
# Delete data
cursor.execute('DELETE FROM users WHERE name = ?', ('Jane Smith',))
conn.commit()
```

## 5. Executing advanced queries

### 5.1. Joins

```python
# Create additional tables
cursor.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        id INTEGER PRIMARY KEY,
        user_id INTEGER,
        product TEXT,
        price REAL
    )
''')

# Insert data into orders table
cursor.execute('INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)', (1, 'Product A', 19.99))
cursor.execute('INSERT INTO orders (user_id, product, price) VALUES (?, ?, ?)', (2, 'Product B', 12.50))

# Perform INNER JOIN between users and orders
cursor.execute('''
    SELECT users.name, orders.product
    FROM users
    INNER JOIN orders ON users.id = orders.user_id
''')
rows = cursor.fetchall()
for row in rows:
    print(row)
```

### 5.2. Aggregates

```python
# Calculate average age of users
cursor.execute('SELECT AVG(age) FROM users')
average_age = cursor.fetchone()[0]
print("Average age:", average_age)

# Count the number of orders for each user
cursor.execute('''
    SELECT users.name, COUNT(orders.id) AS num_orders
    FROM users
    LEFT JOIN orders ON users.id = orders.user_id
    GROUP BY users.name
''')
rows = cursor.fetchall()
for row in rows:
    print(row)
```

### 5.3. Subqueries

```python
# Get users with orders above the average price
cursor.execute('''
    SELECT *
    FROM users
    WHERE id IN (
        SELECT user_id
        FROM orders
        WHERE price > (SELECT AVG(price) FROM orders)
    )
''')
rows = cursor.fetchall()
for row in rows:
    print(row)
```

Remember to commit the changes using `conn.commit()` whenever you modify the database. Also, close the connection and cursor after you're done working with the database:

```python
cursor.close()
conn.close()
```

This covers the basics of using SQLite with Python and executing different types of queries. Always ensure proper error handling and data validation to create robust and secure applications.


In [1]:
import sqlite3

def create_tables(cursor):
    # Create the "category" table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS category (
            categoryid INTEGER PRIMARY KEY,
            cname TEXT NOT NULL
        )
    ''')

    # Create the "movies" table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS movies (
            movieid INTEGER PRIMARY KEY,
            categoryid INTEGER,
            name TEXT NOT NULL,
            year INTEGER,
            duration INTEGER,
            FOREIGN KEY (categoryid) REFERENCES category (categoryid)
        )
    ''')

def insert_data(cursor):
    # Insert data into the "category" table
    query='INSERT INTO category (cname) VALUES (?)'
    cursor.execute(query, ('Action',))
    cursor.execute(query, ('Comedy',))
    cursor.execute(query, ('Drama',))
    cursor.execute(query, ('Science Fiction',))
    cursor.execute(query, ('Fantasy',))

    query='INSERT INTO movies (categoryid, name, year, duration) VALUES (?, ?, ?, ?)'
    # Insert data into the "movies" table
    cursor.execute(query,(1, 'Rush', 2013, 150))
    cursor.execute(query,(1, 'Top Gun: Maverick', 2021, 120))
    cursor.execute(query,(2, 'Americn Pie', 2011, 95))
    cursor.execute(query,(3, 'The Shawshank Redemption', 1994, 135))
    cursor.execute(query,(3, 'Harry Potter:', 2020, 122))

def main():
    # Connect to the database
    conn = sqlite3.connect('movies_list.sqlite3')
    cursor = conn.cursor()

    # Create tables if they don't exist
    create_tables(cursor)

    # Insert some sample data
    insert_data(cursor)

    # Save changes and close the connection
    conn.commit()
    conn.close()

# Execute the function main only if the table data is not present

To access rows by column name instead of indexs of tuple
```python
conn.row_factory=sqlite3.row
```

In [2]:
from contextlib import closing
conn = sqlite3.connect('movies_list.sqlite3')
with closing(conn.cursor()) as cursor:
    cursor.execute("select * from movies")
    rows = cursor.fetchall()  # Fetch all rows
    print("id,cat_id,name,year,duration")
    for row in rows:
        print(row)

id,cat_id,name,year,duration
(1, 1, 'Rush', 2013, 150)
(2, 1, 'Top Gun: Maverick', 2021, 120)
(3, 2, 'Americn Pie', 2011, 95)
(4, 3, 'The Shawshank Redemption', 1994, 135)
(5, 3, 'Harry Potter:', 2020, 122)


In [3]:

with closing(conn.cursor()) as cursor:
    cursor.execute('INSERT INTO movies (categoryid, name, year, duration) VALUES (?, ?, ?, ?)',('1','BArbie','2023','120'))
    conn.commit()
